In [ ]:
%load_ext autoreload
%autoreload 1
%aimport utilsLoaders
%aimport ocfeats


In [ ]:
from pathlib import Path

import pandas as pd # also openpyxl for xlsx files
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm, trange

from utilsLoaders import read_trc, read_mot
import ocfeats

# gdrive = Path('/Volumes/GoogleDrive-112026393729621442608')
gdrive = Path('/Users/psr/Library/CloudStorage/GoogleDrive-paru@stanford.edu')
datadir = gdrive / 'My Drive/NMBL Lab/OpenCap for NMD biomarkers/data'
dataset = '2023-05_dhd'

def get_trc_fpath(sid, trial):
    return datadir / dataset / f'opencap_data/{sid}/' \
        f'MarkerData/PostAugmentation/{trial}/{trial}.trc'

def get_mot_fpath(sid, trial):
    return datadir / dataset / f'opencap_data/{sid}/' \
        f'OpenSimData/Kinematics/{trial}.mot'
    
def get_model_fpath(sid):
    sdir = datadir / dataset / f'opencap_data/{sid}'
    return list((sdir/'OpenSimData/Model').glob('*.osim'))[0]


In [ ]:
# colorblind friendly palette
from cycler import cycler
cp = ["#172A5A", "#FF7171", "#227567", "#34BAEA", "#F9D466", ]
plt.rcParams['axes.prop_cycle'] = cycler(color=cp)

# set default font
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Open Sans', 'Arial']

# automatically despine
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# set defualt DPI
plt.rcParams['figure.dpi'] = 72


In [ ]:
df_session = pd.read_excel(datadir / dataset / 'session_info.xlsx', )
df_trial = pd.read_excel(datadir / dataset / 'trial_info.xlsx')
df_part = pd.read_excel(datadir / dataset / 'participant_info.xlsx')

df_feat = df_part[['pid']].copy()


### Arm ROM

In [ ]:
df_temp = df_trial[df_trial.trial_clean == 'arm_rom']
data = []
for i, row in tqdm(df_temp.iterrows(), total=df_temp.shape[0]):
    pid, sid, trial = row[['pid', 'sid', 'trial']]
    fps, markers, xyz = read_trc(get_trc_fpath(sid, trial))        
    feats = ocfeats.arm_rom_trc_feats(xyz, markers)
    max_mean_sa, mean_ea_at_max_mean_sa, max_min_sa, max_ea_at_max_min_sa, rw = feats

    data.append({'pid': pid,
                 'max_mean_sa': max_mean_sa,
                 'mean_ea_at_max_mean_sa': mean_ea_at_max_mean_sa,
                 'max_min_sa': max_min_sa,
                 'max_ea_at_max_min_sa': max_ea_at_max_min_sa,
                 'reachable_workspace': rw,
                })

df_temp = pd.DataFrame(data)
df_feat.drop(df_temp.columns[1:], axis=1, errors='ignore', inplace=True)
df_feat = df_feat.merge(df_temp, on='pid', how='outer')


### Brooke

In [ ]:
df_temp = df_trial[df_trial.trial_clean == 'brooke']
data = []
for i, row in tqdm(df_temp.iterrows(), total=df_temp.shape[0]):
    pid, sid, trial = row[['pid', 'sid', 'trial']]
    fps, markers, xyz = read_trc(get_trc_fpath(sid, trial))        
    feats = ocfeats.brooke_trc_feats(xyz, markers)
    max_mean_sa, max_mean_ea, max_min_sa, max_ea_at_max_min_sa = feats

    data.append({'pid': pid,
                 'brooke_max_mean_sa': max_mean_sa,
                 'brooke_max_mean_ea': max_mean_ea,
                 'brooke_max_min_sa': max_min_sa,
                 'brooke_max_ea_at_max_min_sa': max_ea_at_max_min_sa,
                })

df_temp = pd.DataFrame(data)
df_feat.drop(df_temp.columns[1:], axis=1, errors='ignore', inplace=True)
df_feat = df_feat.merge(df_temp, on='pid', how='outer')


### Curls

In [ ]:
df_temp = df_trial[df_trial.trial_clean == 'curls']
data = []
for i, row in tqdm(df_temp.iterrows(), total=df_temp.shape[0]):
    pid, sid, trial = row[['pid', 'sid', 'trial']]
    fps, markers, xyz = read_trc(get_trc_fpath(sid, trial))        
    max_mean_ea = ocfeats.curls_trc_feats(xyz, markers)

    data.append({'pid': pid,
                 'curls_max_mean_ea': max_mean_ea,
                })

df_temp = pd.DataFrame(data)
df_feat.drop(df_temp.columns[1:], axis=1, errors='ignore', inplace=True)
df_feat = df_feat.merge(df_temp, on='pid', how='outer')


### 10 Meter Walk

In [ ]:
df_temp = df_trial[df_trial.trial_clean == '10mwt']
data = []
for i, row in tqdm(df_temp.iterrows(), total=df_temp.shape[0]):
    pid, sid, trial = row[['pid', 'sid', 'trial']]
    
    fps, markers, xyz = read_trc(get_trc_fpath(sid, trial))        
    feats = ocfeats.gait_trc_feats(xyz, markers, fps)
    time_10m, speed, com_bob, com_sway, last_4m = feats
    
    df = read_mot(get_mot_fpath(sid, trial))
    feats = ocfeats.gait_mot_feats(df, last_4m)
    mean_ptp_hip_add, mean_max_ka = feats

    data.append({'pid': pid,
                 'walk_speed': speed,
                 'walk_com_sway': com_sway,
                 'walk_mean_ptp_hip_add': mean_ptp_hip_add,
                 'walk_mean_max_ka': mean_max_ka,
                })

df_temp = pd.DataFrame(data)
df_feat.drop(df_temp.columns[1:], axis=1, errors='ignore', inplace=True)
df_feat = df_feat.merge(df_temp, on='pid', how='outer')


### 10 Meter Walk/Run

In [ ]:
df_temp = df_trial[df_trial.trial_clean == '10mwrt']
data = []
for i, row in tqdm(df_temp.iterrows(), total=df_temp.shape[0]):
    pid, sid, trial = row[['pid', 'sid', 'trial']]
    
    fps, markers, xyz = read_trc(get_trc_fpath(sid, trial))        
    feats = ocfeats.gait_trc_feats(xyz, markers, fps)
    time_10m, speed, com_bob, com_sway, last_4m = feats
    
    df = read_mot(get_mot_fpath(sid, trial))
    feats = ocfeats.gait_mot_feats(df, last_4m)
    mean_ptp_hip_add, mean_max_ka = feats

    data.append({'pid': pid,
                 'run_speed': speed,
                 'run_com_sway': com_sway,
                 'run_mean_ptp_hip_add': mean_ptp_hip_add,
                 'run_mean_max_ka': mean_max_ka,
                })

df_temp = pd.DataFrame(data)
df_feat.drop(df_temp.columns[1:], axis=1, errors='ignore', inplace=True)
df_feat = df_feat.merge(df_temp, on='pid', how='outer')


### Jump

In [ ]:
df_temp = df_trial[df_trial.trial_clean == 'jump']
data = []
for i, row in tqdm(df_temp.iterrows(), total=df_temp.shape[0]):
    pid, sid, trial = row[['pid', 'sid', 'trial']]
    
    fps, markers, xyz = read_trc(get_trc_fpath(sid, trial))  
    com_xyz = ocfeats.center_of_mass(get_model_fpath(sid),
                                     get_mot_fpath(sid, trial))
    max_com_vel = ocfeats.jump_trc_feats(com_xyz, fps)

    data.append({'pid': pid,
                 'jump_max_com_vel': max_com_vel,
                })

df_temp = pd.DataFrame(data)
df_feat.drop(df_temp.columns[1:], axis=1, errors='ignore', inplace=True)
df_feat = df_feat.merge(df_temp, on='pid', how='outer')


### Toe Stand

In [ ]:
df_temp = df_trial[df_trial.trial_clean == 'toe_stand']
data = []
for i, row in tqdm(df_temp.iterrows(), total=df_temp.shape[0]):
    pid, sid, trial = row[['pid', 'sid', 'trial']]
    
    df = read_mot(get_mot_fpath(sid, trial))        
    mean_int_aa = ocfeats.toe_stand_mot_feats(df)
    
    fps, markers, xyz = read_trc(get_trc_fpath(sid, trial))        
    com_xyz = ocfeats.center_of_mass(get_model_fpath(sid),
                                     get_mot_fpath(sid, trial))
    feats = ocfeats.toe_stand_trc_feats(xyz, markers, fps, com_xyz)
    int_com_elev, int_com_fwd, int_mean_heel_elev = feats

    data.append({'pid': pid,
                 'toe_stand_mean_int_aa': mean_int_aa,
                 'toe_stand_int_com_elev': int_com_elev,
                 'toe_stand_int_com_fwd': int_com_fwd,
                 'toe_stand_int_mean_heel_elev': int_mean_heel_elev,
                })

df_temp = pd.DataFrame(data)
df_feat.drop(df_temp.columns[1:], axis=1, errors='ignore', inplace=True)
df_feat = df_feat.merge(df_temp, on='pid', how='outer')


In [ ]:
df_feat.to_pickle('./feats.pkl')
